In [2]:
import fitz  # PyMuPDF
import cv2
import os
from ultralytics import YOLO
from PIL import Image

# Пути
pdf_path = "C:\\Users\\Vito\\Jupyter\\Генератор\\2404.19737v1.pdf"  # Исходный PDF
output_dir = "C:\\Users\\Vito\\Jupyter\\Генератор\\Новая папка"  # Папка для сохранения изображений
output_pdf_path = "C:\\Users\\Vito\\Jupyter\\Генератор\\annotated_document1.pdf"  # Итоговый PDF
os.makedirs(output_dir, exist_ok=True)

# Загружаем модель YOLO
model = YOLO("C:\\Users\\Vito\\Jupyter\\Models\\runs YOLO11s datasetv3 aug imgsz=960\\detect\\train\\weights\\best.pt")

# 1. Конвертируем PDF в изображения
pdf_document = fitz.open(pdf_path)
image_paths = []

for page_number in range(len(pdf_document)):
    # Извлекаем страницу как изображение
    page = pdf_document[page_number]
    pix = page.get_pixmap(dpi=300)  # Указываем качество изображения (300 DPI)
    image_path = os.path.join(output_dir, f"page_{page_number + 1}.png")
    pix.save(image_path)
    image_paths.append(image_path)

pdf_document.close()

# Настройка кастомных порогов для каждого класса
custom_confidences = {
    0: 0.3,  # title
    1: 0.1,  # paragraph
    9: 0.15, # footer
    11: 0.1,  # formula
    10: 0.88, # footnote
    6: 0.9, # numbered_list
    7: 0.9 # marked_list
}

# 2. Обрабатываем изображения моделью YOLO и визуализируем предсказания с кастомными порогами
annotated_images = []
for image_path in image_paths:
    # Читаем изображение
    img = cv2.imread(image_path)
    
    # Применяем модель YOLO
    results = model.predict(source=img, conf=0.4, iou=0.51, save=False)
    
    # Фильтруем предсказания на основе кастомных порогов для классов
    filtered_boxes = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls)  # Класс объекта
            conf = float(box.conf)  # Уверенность объекта

            # Проверяем, превышает ли уверенность кастомный порог для данного класса
            if cls in custom_confidences:
                if conf >= custom_confidences[cls]:
                    filtered_boxes.append(box)
            else:
                # Используем общий порог (например, 0.4) для классов, для которых не задан кастомный порог
                if conf >= 0.4:
                    filtered_boxes.append(box)

        # Применяем отфильтрованные боксы для визуализации
        result.boxes = filtered_boxes

    # Наносим рамки на изображение
    annotated_img = results[0].plot()  # Визуализируем предсказания (аннотированное изображение)
    annotated_image_path = image_path.replace(".png", "_annotated.png")
    cv2.imwrite(annotated_image_path, annotated_img)
    annotated_images.append(annotated_image_path)

# 3. Преобразуем аннотированные изображения обратно в PDF
images = [Image.open(img).convert("RGB") for img in annotated_images]
images[0].save(output_pdf_path, save_all=True, append_images=images[1:])

print(f"Обработанный PDF сохранён в {output_pdf_path}")


0: 960x768 3 paragraphs, 1 picture, 1 footer, 1 footnote, 287.3ms
Speed: 24.7ms preprocess, 287.3ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 5 paragraphs, 1 picture, 1 table_signature, 1 marked_list, 1 footer, 235.0ms
Speed: 8.3ms preprocess, 235.0ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 4 paragraphs, 1 table, 1 picture, 3 numbered_lists, 1 footer, 261.6ms
Speed: 0.0ms preprocess, 261.6ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 1 title, 4 paragraphs, 1 table, 1 footer, 1 footnote, 262.8ms
Speed: 19.4ms preprocess, 262.8ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 4 paragraphs, 2 pictures, 1 numbered_list, 1 footer, 267.2ms
Speed: 0.0ms preprocess, 267.2ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 5 paragraphs, 2 pictures, 1 footer, 356.9ms
Speed: 22.3ms preprocess, 356.9ms inference, 0.0ms postprocess per 